In [1]:
import torch
import multiprocessing
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import wandb
import numpy as np
import random

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
multiprocessing.set_start_method("spawn", force=True)

In [4]:
wandb.login(key="c62c25918ce7fcb2ae41fc887c02e43aeb0cb261")  

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: challesvu (challesvu-ton-duc-thang-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

In [6]:
train_data = load_data('/kaggle/input/belong-irf/train.csv')
val_data = load_data('/kaggle/input/belong-irf/val.csv')

In [7]:
class sDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [10]:
def encode_labels(labels):
    return ["Yes" if label == "Yes" else "No" for label in labels]

train_texts = train_data['post'].tolist()
train_labels = encode_labels(train_data['is_belong'].tolist())
val_texts = val_data['post'].tolist()
val_labels = encode_labels(val_data['is_belong'].tolist())

In [11]:
label_map = {"No": 0, "Yes": 1}
train_indices = [label_map[label] for label in train_labels]
val_indices = [label_map[label] for label in val_labels]
train_dataset = sDataset(train_texts, train_indices, tokenizer, max_len=512)
val_dataset = sDataset(val_texts, val_indices, tokenizer, max_len=512)

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    run_name="mental-bert-finetune",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    disable_tqdm=False,
    dataloader_num_workers=0
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [15]:
trainer.train()

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241226_172358-4rkolnxy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run mental-bert-finetune
wandb: ⭐️ View project at https://wandb.ai/challesvu-ton-duc-thang-university/huggingface
wandb: 🚀 View run at https://wandb.ai/challesvu-ton-duc-thang-university/huggingface/runs/4rkolnxy
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.537100,0.446192,0.805274,0.833910,0.806020,0.863799
2,0.431400,0.435161,0.815416,0.848586,0.791925,0.913978
3,0.260600,0.428793,0.823529,0.840367,0.860902,0.820789
4,0.201600,0.533558,0.831643,0.851521,0.850000,0.853047
5,0.107200,0.811142,0.825558,0.856187,0.802508,0.917563
6,0.036800,0.892339,0.829615,0.846154,0.865169,0.827957
7,0.028800,1.125193,0.829615,0.850000,0.846975,0.853047


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=434, training_loss=0.24032902434330933, metrics={'train_runtime': 883.4172, 'train_samples_per_second': 44.645, 'train_steps_per_second': 1.404, 'total_flos': 3631985008189440.0, 'train_loss': 0.24032902434330933, 'epoch': 7.0})

In [16]:
model.save_pretrained("mental-roberta-finetune-irf")
tokenizer.save_pretrained("mental-roberta-finetune-irf")

('mental-roberta-finetune-irf/tokenizer_config.json',
 'mental-roberta-finetune-irf/special_tokens_map.json',
 'mental-roberta-finetune-irf/vocab.json',
 'mental-roberta-finetune-irf/merges.txt',
 'mental-roberta-finetune-irf/added_tokens.json')

## Evaluate

In [17]:
test_data = load_data('/kaggle/input/belong-irf/test.csv')

In [18]:
test_texts = test_data['post'].tolist()
test_labels = encode_labels(test_data['is_belong'].tolist())

In [19]:
test_indices = [label_map[label] for label in test_labels]
test_dataset = sDataset(test_texts, test_indices, tokenizer, max_len=512)

In [20]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [21]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8430
Precision: 0.8429
Recall: 0.8803
F1 Score: 0.8612
